In [1]:
# Importando as bibliotecas que iremos utilizar:
import nltk
import re
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split

In [2]:
# ----------- Início da Declaração de funções ---------------
def marque_negacao(texto):
    negacoes = ['não','not']
    negacao_detectada = False
    resultado = []
    palavras = texto.split()
    for p in palavras:
        p = p.lower()
        if negacao_detectada == True:
            p = p + '_NEG'
        if p in negacoes:
            negacao_detectada = True
        resultado.append(p)
    return (" ".join(resultado))

def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','')
    instancia = remove_emoji(instancia)
    instancia = Lemmatization(instancia)
    instancia  = Stemming(instancia)
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

def Lemmatization(instancia):
    wordnet_lemmatizer = WordNetLemmatizer()
    palavras = []
    for w in instancia.split():
        palavras.append(wordnet_lemmatizer.lemmatize(w))
    return (" ".join(palavras))

def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

# Aplicando o stemming em nossa base:
def Stemming(instancia):
    stemmer = nltk.stem.snowball.EnglishStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

# Função para remover Stopwords da nossa base:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Função para remover os emojis do texto by: https://stackoverflow.com/a/49146722/330558 &
# https://www.reddit.com/r/learnpython/comments/8br5sz/removing_emojis_from_words_python3/dx91wrm/
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "\U00010000-\U0010ffff"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

# Função para mostrar os resultados do classificador
def Metricas(modelo, tweets, classes):
    resultados = cross_val_predict(modelo, tweets, classes, cv=10)
    # Matriz de confusão:
    print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))
    print (metrics.classification_report(classes,resultados))
    return 'Acurácia do modelo: {}'.format(metrics.accuracy_score(classes,resultados))

# ----------- Final da Declaração de funções ---------------

In [3]:
# abre o arquivo correspondente
#RODAR ESSA CELULA APENAS SE NÃO TIVER O ARQUIVO CSV DA DATABASE
file = open("DB\\todos_comentarios.txt", 'r', encoding="UTF-8")
# cria um pandas.DataFrame para armazena os resultados
df = pd.DataFrame(columns=["Avaliação", "Comentário"])
linha = file.readline()

while linha != "":
    linhaSep = linha.split(",", maxsplit=1)
    # separa toda a string da avaliação em uma lista, em que cada index é uma avaliação
    Avalliation = linhaSep[0]
    Comment = linhaSep[1]
    # escreve em cada coluna do arquivo a sua avaliação e seu respectivo comentário.
    df = df.append({"Avaliação": Avalliation, "Comentário": Comment}, ignore_index=True)
    linha = file.readline()  # manuntenção do loop

df.to_csv("todos_comentarios.csv", index=False)  # salvando o csv
print("Arquivos Formatados para csv")


FileNotFoundError: [Errno 2] No such file or directory: 'DB\\todos_comentarios.txt'

In [4]:
# Lendo a base de dados:
df = pd.read_csv('todos_comentarios.csv', encoding='utf-8')

In [5]:
# Verificando novamente o número de linhas dessa coluna:
df.Comentário.count()

31633

In [6]:
tweets = df['Comentário']
classes = df['Avaliação']

In [7]:
tweets = [Preprocessing(i) for i in tweets]

In [8]:
tweet_tokenizer = TweetTokenizer() 

In [9]:
# Instancia o objeto que faz a vetorização dos dados de texto:
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [10]:
freq_tweets = vectorizer.fit_transform(tweets)
type(freq_tweets)

scipy.sparse.csr.csr_matrix

In [ ]:
print(tweets[0]+"\n"+freq_tweets[0]+"\n")

In [11]:
freq_tweets.shape

(31633, 28992)

In [12]:
#Rodar essa celula se quiser OVERsampling
bl = SMOTE()
freq_tweets,classes = bl.fit_resample(freq_tweets, classes)
freq_tweets.shape

(57220, 28992)

In [12]:
#Rodar essa celula se quiser UNDERsampling
bl = NearMiss()
freq_tweets,classes = bl.fit_resample(freq_tweets, classes)
freq_tweets.shape

(15680, 28992)

In [13]:
#Classificador NB
nbClassifier = Pipeline([('classifier', MultinomialNB())])

In [14]:
#Classificador SVM:
pipeline_svm_simples = Pipeline([('classifier', svm.SVC())])

In [15]:
#Classificador LR
pipeline_LR = Pipeline([('classifier',LogisticRegression(random_state=0, solver='newton-cg') )])

In [17]:
# Pipeline que atribui tag de negações nas palavras:
pipeline_negacoes = Pipeline([('counts', CountVectorizer(tokenizer=lambda text: marque_negacao(text))),('classifier', MultinomialNB())])

In [18]:
# Pipeline com tag de negação:
pipeline_svm_negacoes = Pipeline([('counts', CountVectorizer(tokenizer=lambda text: marque_negacao(text))),('classifier', svm.SVC(kernel='linear'))])

In [19]:
# Metricas do Naive Bayes simples:
Metricas(nbClassifier,freq_tweets,classes)

Predito     1     2     3     4     5    All
Real                                        
1         907  1549   401   166   113   3136
2         102  1732  1102   179    21   3136
3          53  1556  1066   398    63   3136
4          31   917   890  1146   152   3136
5          57   270   359  1160  1290   3136
All      1150  6024  3818  3049  1639  15680
              precision    recall  f1-score   support

           1       0.79      0.29      0.42      3136
           2       0.29      0.55      0.38      3136
           3       0.28      0.34      0.31      3136
           4       0.38      0.37      0.37      3136
           5       0.79      0.41      0.54      3136

    accuracy                           0.39     15680
   macro avg       0.50      0.39      0.40     15680
weighted avg       0.50      0.39      0.40     15680



'Acurácia do modelo: 0.3916454081632653'

In [20]:
# Metricas do LR simples:
Metricas(pipeline_LR,freq_tweets,classes)

Predito     1     2     3     4     5    All
Real                                        
1        2095   278   231   180   352   3136
2         493  1133  1023   400    87   3136
3         337   898   989   661   251   3136
4         209   362   612  1354   599   3136
5         222    12    61   285  2556   3136
All      3356  2683  2916  2880  3845  15680
              precision    recall  f1-score   support

           1       0.62      0.67      0.65      3136
           2       0.42      0.36      0.39      3136
           3       0.34      0.32      0.33      3136
           4       0.47      0.43      0.45      3136
           5       0.66      0.82      0.73      3136

    accuracy                           0.52     15680
   macro avg       0.50      0.52      0.51     15680
weighted avg       0.50      0.52      0.51     15680



'Acurácia do modelo: 0.5183035714285714'

In [21]:
# Metricas do SVM linear simples:
Metricas(pipeline_svm_simples,freq_tweets,classes)

Predito     1     2     3     4     5    All
Real                                        
1        2195    95   150   183   513   3136
2         504  1628   612   291   101   3136
3         426  1215   640   556   299   3136
4         297   331   524  1311   673   3136
5         236     0    16   135  2749   3136
All      3658  3269  1942  2476  4335  15680
              precision    recall  f1-score   support

           1       0.60      0.70      0.65      3136
           2       0.50      0.52      0.51      3136
           3       0.33      0.20      0.25      3136
           4       0.53      0.42      0.47      3136
           5       0.63      0.88      0.74      3136

    accuracy                           0.54     15680
   macro avg       0.52      0.54      0.52     15680
weighted avg       0.52      0.54      0.52     15680



'Acurácia do modelo: 0.5435586734693878'

In [22]:
# Naive Bayes com tag de negacoes:
Metricas(pipeline_negacoes,tweets,classes)

ValueError: Found input variables with inconsistent numbers of samples: [31633, 15680]

In [23]:
# SVM linear com tag de negacoes:
Metricas(pipeline_svm_negacoes,tweets,classes)

ValueError: Found input variables with inconsistent numbers of samples: [31633, 15680]

In [ ]:
# Cross validation:
resultados = cross_val_predict(pipeline_negacoes, tweets, classes, cv=10)